In [1]:
# yelp scraping project
import requests
import json
import pandas as pd

from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
# create input variables

search = input("What would you like to search: ")
zipcode = input("Where would you like to search: ")

# changed where to Where
# created space after search:" to search: "

What would you like to search: Barbers
Where would you like to search: 90742


In [3]:
# default web search to concatenate
default = "https://www.yelp.com/search?find_desc="
locationUrl = "&find_loc="
end = "&ns=1"

# empty lists to append to
url = []
business_name = []
business_category = []
yelp_rating = []
review_count = []
price_range = []
price_category = []

address = []
phone = []
website = []

# new data
review_content = []

In [4]:
# function to search input parameters on Yelp
def search_yelp(search, zipcode):
    
    keyword = search;
    zipcode = zipcode;
    query_url = f"{default}{search}{locationUrl}{zipcode}{end}";
    response = requests.get(query_url)
    soup = BeautifulSoup(response.text, 'html.parser');
    for link in soup.find_all('a', class_="biz-name"):
        url.append("https://www.yelp.com"+link.get('href'))
    url.pop(0)

In [5]:
# function to scrape yelp data on each page
def scrape_yelp(url):
    for i in url:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        business_name.append(soup.find('h1', attrs={'class':'biz-page-title'}).text.strip())
        business_category.append(soup.find(attrs={'class': 'category-str-list'}).text.strip())
        yelp_rating.append(soup.find(attrs={"class":"i-stars"})["title"])
        review_count.append(soup.find(attrs={'class': 'review-count rating-qualifier'}).text.strip())
        price_range.append(soup.find(attrs={'class': 'business-attribute price-range'}).text.strip())
        price_category.append(soup.find(attrs={'class': 'nowrap price-description'}).text.strip())
        address.append(soup.find(attrs={'class': 'street-address'}).text.strip())
        phone.append(soup.find(attrs={'class': 'biz-phone'}).text.strip())
        
        if soup.find("a", href=lambda href: href and "biz_redir?" in href) is not None:
            website.append(soup.find("a", href=lambda href: href and "biz_redir?" in href).text.strip())
        else:
            website.append('no website')       

In [6]:
# create a dataframe with yelp data from yelp scrape

def create_table():
    data = {
    'BusinessName' : business_name,
    'BusinessCategory' : business_category,
    'YelpRating' : yelp_rating,
    'ReviewCount' : review_count,
    'PriceRange($)' : price_range,
    'PriceCategory': price_category,
    'Address' : address, 
    'Phone' : phone,
    'Website' : website
        
    }
    # return data
    df = pd.DataFrame.from_dict(data)
    df = df[['BusinessName','BusinessCategory','YelpRating','ReviewCount','PriceRange($)', 'PriceCategory','Address',
            'Phone', 'Website']]
    df.to_csv(f'data/yelp_{search}{zipcode}.csv')
    return df

In [7]:
# RUN FUNCTION
search_yelp(search, zipcode)

In [8]:
# RUN FUNCTION
scrape_yelp(url)

In [9]:
# RUN FUNCTION
create_table()

,BusinessName,BusinessCategory,YelpRating,ReviewCount,PriceRange($),PriceCategory,Address,Phone,Website
0,Harbor Barber,Barbers,5.0 star rating,178 reviews,$$,Moderate,"17212 Pacific Coast HwySte BHuntington Beach, ...",(714) 907-0232,harborbarberhb.com
1,Main Street Barber Shop,Barbers,4.5 star rating,96 reviews,$$,Moderate,"317 1/2 Main StSeal Beach, CA 90740",(562) 430-0236,no website
2,Springdale Barbers,Barbers,4.5 star rating,52 reviews,$,Inexpensive,"15792 Springdale StHuntington Beach, CA 92649",(714) 901-7110,no website
3,Hair Safari,Barbers,5.0 star rating,58 reviews,$,Inexpensive,"5931 Warner AveHuntington Beach, CA 92649",(714) 840-1877,hairsafari.com
4,Marina Clipper Barber Shop,Barbers,5.0 star rating,1 review,$$,Moderate,"1081 Pacific Coast HwySeal Beach, CA 90740",(562) 431-9369,no website
5,Mario’s Sports Barbershop,Barbers,4.0 star rating,22 reviews,$,Inexpensive,"14900 Springdale StHuntington Beach, CA 92647",(714) 901-2350,mariossportsbarbershop.com
6,The End Result,"Barbers,\n Hair Salons",4.5 star rating,12 reviews,$$,Moderate,"14022 Rancho RdWestminster, CA 92683",(714) 892-0313,no website
7,Studio Fifty Fifty,"Hair Salons,\n Nail Salons,...",4.5 star rating,390 reviews,$$,Moderate,"5050 Heil AveHuntington Beach, CA 92649",(714) 840-9777,studiofiftyfifty.com
8,Salon M,Hair Salons,4.5 star rating,129 reviews,$$,Moderate,"16561 Bolsa Chica StSte 203Huntington Beach, C...",(714) 377-1000,salon-m.net
9,Salon Meritage,"Hair Salons,\n Eyebrow Serv...",5.0 star rating,49 reviews,$$,Moderate,"500 E Pacific Coast HwySte 100Seal Beach, CA 9...",(562) 493-2639,salonmeritage.com
